In [5]:
import pandas as pd
import numpy as np
import datetime as dt


In [4]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


link="https://drive.google.com/open?id=1tAIcp18-PMaVJku8nkmcqURdIkwyXRWI"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('quote.csv')  
data=pd.read_csv('quote.csv')[['DATE','TIME_M','BID','BIDSIZ','ASK','ASKSIZ']]
data['TIME_M']=data['TIME_M'].apply(lambda x: x[:-10])


You should consider upgrading via the 'c:\users\naman\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'google.colab'

In [0]:
# import quote data
# data=pd.read_csv('raw_data/quote.csv')[['DATE','TIME_M','BID','BIDSIZ','ASK','ASKSIZ']]
# data['TIME_M']=data['TIME_M'].apply(lambda x: x[:-10])

In [ ]:
# data cleaning
ASK=data.groupby(['DATE','TIME_M']).apply(lambda grp: grp.groupby('ASK')['ASKSIZ'].sum().reset_index())
print(1)
ASK=ASK[ASK['ASKSIZ']>=10]
ASKg=ASK.groupby(['DATE','TIME_M'])
print(2)
BID=data.groupby(['DATE','TIME_M']).apply(lambda grp: grp.groupby('BID')['BIDSIZ'].sum().reset_index())
print(3)
BID=BID[BID['BIDSIZ']>=10]
BIDg=BID.groupby(['DATE','TIME_M'])

In [0]:
import pickle
# pickle.dump(ASK, open( "clean_data/ASK.p", "wb" ) )
# pickle.dump(BID, open( "clean_data/BID.p", "wb" ) )
# pickle.dump(ASKg, open( "clean_data/ASKg.p", "wb" ) )
# pickle.dump(BIDg, open( "clean_data/BIDg.p", "wb" ) )

ASK=pickle.load(open("clean_data/ASK.p",'rb'))
BID=pickle.load(open("clean_data/BID.p",'rb'))
ASKg=pickle.load(open("clean_data/ASKg.p",'rb'))
BIDg=pickle.load(open("clean_data/BIDg.p",'rb'))



In [0]:
# feature generating
X=pd.DataFrame()
X['ask_bid_ratio']=((ASKg['ASKSIZ'].sum()-BIDg['BIDSIZ'].sum()) /BIDg['BIDSIZ'].sum()).dropna()
X['ask_bid_spread']=((ASKg['ASK'].mean()-BIDg['BID'].mean()) /BIDg['BID'].mean()).dropna()
X['time']=(pd.to_datetime(X.index.get_level_values(1),format='%H:%M:%S')-dt.datetime(1900,1,1,9,30)).total_seconds()/(3600*6.5)

va_ask_mean=(ASK['ASK']*ASK['ASKSIZ']).groupby(['DATE','TIME_M']).sum()/ASKg['ASKSIZ'].sum()
va_bid_mean=(BID['BID']*BID['BIDSIZ']).groupby(['DATE','TIME_M']).sum()/BIDg['BIDSIZ'].sum()
X['ask_skew']=(va_ask_mean-ASKg['ASK'].mean())/ASKg['ASK'].mean()
X['bid_skew']=(va_bid_mean-BIDg['BID'].mean())/BIDg['BID'].mean()

In [0]:
# import trade data
link="https://drive.google.com/open?id=1Vev9OAKiRjEfadwIcVMNvsK8dRMaJlvk"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('trade.csv')  
data=pd.read_csv(trade.csv')[['DATE','TIME_M','PRICE','SIZE']]
data['TIME_M']=data['TIME_M'].apply(lambda x: x[:-10])

In [0]:
#data cleaning, compute volume weighted price
data['amt']=data['SIZE']*data['PRICE']
prices=data.groupby(['DATE','TIME_M'])['amt'].sum()/data.groupby(['DATE','TIME_M'])['SIZE'].sum()

In [0]:
# generate Y
Y=pd.DataFrame({'jump': 1*(abs(prices.diff())>(2*prices.rolling(3600).std()))})

In [0]:
df=X.merge(Y,left_index=True, right_index=True)

In [0]:
df['jump_lag_1']=df['jump'].shift().fillna(0)
df.to_csv('data.csv')
df

ask_bid_ratio  ask_bid_spread      time  ask_skew  \
DATE     TIME_M                                                        
20200102 10:00:00      -0.128755        0.134638  0.076923 -0.000002   
         10:00:01       0.022901        0.134638  0.076966 -0.000006   
         10:00:02      -0.292308        0.134638  0.077009  0.000011   
         10:00:03      -0.503448        0.134597  0.077051  0.000000   
         10:00:04      -0.397938        0.097669  0.077094 -0.000014   
...                          ...             ...       ...       ...   
20200131 9:59:55        0.033195        0.010947  0.076709 -0.002951   
         9:59:56       -0.228916        0.000381  0.076752  0.000001   
         9:59:57        0.400000        0.000448  0.076795  0.000000   
         9:59:58       -0.051724        0.000537  0.076838  0.000005   
         9:59:59        0.000000        0.000336  0.076880 -0.000010   

                   bid_skew  jump  jump_lag_1  
DATE     TIME_M                                
20200102 10:00:00  0.108434     0         0.0  
         10:00:01  0.088216     0         0.0  
         10:00:02  0.041348     0         0.0  
         10:00:03  0.050966     0         0.0  
         10:00:04  0.085395     0         0.0  
...                     ...   ...         ...  
20200131 9:59:55   0.003368     0         0.0  
         9:59:56   0.000081     0         0.0  
         9:59:57   0.000000     0         0.0  
         9:59:58  -0.000043     0         0.0  
         9:59:59  -0.000016     0         0.0  

[287849 rows x 7 columns]